In [1]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for wordcloud making and image importing
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image

# And libraries for data transformation
import datetime

#words counter
from collections import Counter
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import spacy
from nltk.tokenize import word_tokenize


# sentiment 
from textblob import TextBlob


In [2]:
# load data
files = ['csv/2020-01-final.csv', 'csv/2020-02-final.csv', 'csv/2020-03-final.csv', 'csv/2020-04-final.csv', 'csv/2020-05-final.csv']
# read files
df_month = [pd.read_csv(i) for i in files]
# concatenate files
df = pd.concat(df_month)

/Users/allen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15205667 entries, 0 to 960492
Data columns (total 8 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   id_str              object
 1   created_at          object
 2   state               object
 3   sentiment           object
 4   text_clean          object
 5   afn_sentiment       object
 6   vader_sentiment     object
 7   ensemble_sentiment  object
dtypes: object(8)
memory usage: 1.0+ GB


In [4]:
df.isnull().sum(axis = 0)

id_str                      0
created_at                210
state                 1040415
sentiment                   0
text_clean             105700
afn_sentiment          105700
vader_sentiment        105700
ensemble_sentiment          0
dtype: int64

In [5]:
# delete the null row by 'text_clean'
df = df[df['text_clean'].notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15099967 entries, 0 to 960492
Data columns (total 8 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   id_str              object
 1   created_at          object
 2   state               object
 3   sentiment           object
 4   text_clean          object
 5   afn_sentiment       object
 6   vader_sentiment     object
 7   ensemble_sentiment  object
dtypes: object(8)
memory usage: 1.0+ GB


In [6]:
df.isnull().sum(axis = 0)

id_str                      0
created_at                210
state                 1032616
sentiment                   0
text_clean                  0
afn_sentiment               0
vader_sentiment             0
ensemble_sentiment          0
dtype: int64

In [7]:
# delete the null row by 'state'
df = df[df['state'].notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14067351 entries, 0 to 960492
Data columns (total 8 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   id_str              object
 1   created_at          object
 2   state               object
 3   sentiment           object
 4   text_clean          object
 5   afn_sentiment       object
 6   vader_sentiment     object
 7   ensemble_sentiment  object
dtypes: object(8)
memory usage: 965.9+ MB


In [8]:
import re, datetime
def extract_date(x):
#     match = re.search('\d{2}-\d{2}', x)
#     date = datetime.datetime.strptime(match.group(), '%m-%d').date()
    x_list = x.split(' ')
    month = '1' if x_list[1] == 'Jan' else '2' if x_list[1] == 'Feb' else '3' if x_list[1] =='Mar' else '4' if x_list[1] == 'Apr' else '5' if x_list[1] == 'May' else'others'
    if len(x_list[2]) == 1:
        x_list[2] = '0' + x_list[2]
    date = month + '/' + x_list[2] + '/' + '20'
    return date
df['date'] = df['created_at'].apply(lambda x: extract_date(x))
df.head()

,id_str,created_at,state,sentiment,text_clean,afn_sentiment,vader_sentiment,ensemble_sentiment,date
0,1219856157608566784,Wed Jan 22 05:35:46 +0000 2020,California,positive,momsdemand brendaof fun fact accidental death...,negative,positive,neutral,1/22/20
2,1219856816114487296,Wed Jan 22 05:38:23 +0000 2020,Wisconsin,positive,troopax johnmaamd ravin coqundoodledoo edwardh...,negative,negative,negative,1/22/20
3,1219856906270990336,Wed Jan 22 05:38:44 +0000 2020,Pennsylvania,negative,brutal season protect yourself,negative,negative,negative,1/22/20
4,1219857010696630272,Wed Jan 22 05:39:09 +0000 2020,New York,positive,wish administrators cdc could impeached failin...,negative,negative,negative,1/22/20
6,1219858026242416640,Wed Jan 22 05:43:11 +0000 2020,New Jersey,positive,united states first confirmed case new virus a...,negative,negative,negative,1/22/20


In [9]:
# target sentiment
df['sentiment'] = df['ensemble_sentiment']

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14067351 entries, 0 to 960492
Data columns (total 9 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   id_str              object
 1   created_at          object
 2   state               object
 3   sentiment           object
 4   text_clean          object
 5   afn_sentiment       object
 6   vader_sentiment     object
 7   ensemble_sentiment  object
 8   date                object
dtypes: object(9)
memory usage: 1.0+ GB


# 1. Data Visualization

In [11]:
from settings import states_full_dic
df['state_abb'] = df['state'].apply(lambda x: states_full_dic[x] if x is not np.nan else x)
df.head()

,id_str,created_at,state,sentiment,text_clean,afn_sentiment,vader_sentiment,ensemble_sentiment,date,state_abb
0,1219856157608566784,Wed Jan 22 05:35:46 +0000 2020,California,neutral,momsdemand brendaof fun fact accidental death...,negative,positive,neutral,1/22/20,CA
2,1219856816114487296,Wed Jan 22 05:38:23 +0000 2020,Wisconsin,negative,troopax johnmaamd ravin coqundoodledoo edwardh...,negative,negative,negative,1/22/20,WI
3,1219856906270990336,Wed Jan 22 05:38:44 +0000 2020,Pennsylvania,negative,brutal season protect yourself,negative,negative,negative,1/22/20,PA
4,1219857010696630272,Wed Jan 22 05:39:09 +0000 2020,New York,negative,wish administrators cdc could impeached failin...,negative,negative,negative,1/22/20,NY
6,1219858026242416640,Wed Jan 22 05:43:11 +0000 2020,New Jersey,negative,united states first confirmed case new virus a...,negative,negative,negative,1/22/20,NJ


# 2. Correlation analysis

## 2.1 Correlation of total tweets and cases for each state

In [12]:
df_us = pd.read_csv('csv/time_series_covid19_confirmed_US.csv')
df_us.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,145,149,149,151,151,151,151,152,152,152
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,15,15,15,16,16,19,19,19,19
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1924,1968,2031,2156,2173,2198,2256,2299,2329,2427
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,66,66,66,68,68,69,69,69,69,69


In [13]:
df_states = df_us.groupby('Province_State').sum().reset_index()
df_states.head()

,Province_State,UID,code3,FIPS,Lat,Long_,1/22/20,1/23/20,1/24/20,1/25/20,...,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20
0,Alabama,5796241491,57960,241491.0,2203.246784,-5809.578199,0,0,0,0,...,8437,8691,9046,9385,9668,9889,10164,10464,10700,11101
1,Alaska,2604232344,26040,232344.0,1747.579877,-4229.319334,0,0,0,0,...,371,372,374,377,378,379,379,383,383,383
2,American Samoa,16,16,60.0,-14.271000,-170.132000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arizona,1428230216,14280,230216.0,505.138555,-1671.948482,0,0,0,0,...,9305,9707,9945,10526,10960,11119,11383,11736,12216,12674
4,Arkansas,6468550635,64680,550635.0,2618.391704,-6932.548370,0,0,0,0,...,3525,3611,3703,3747,3747,4012,4043,4164,4236,4366


In [14]:
from settings import states_full_dic
df_date = df_states.drop(columns=['UID', 'code3', 'FIPS', 'Lat', 'Long_'])
df_date['state'] = df_states['Province_State']
df_date['state_abb'] = df_date['state'].apply(lambda x: states_full_dic[x] if x in states_full_dic else np.nan)
df_date.head()

,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,state,state_abb
0,Alabama,0,0,0,0,0,0,0,0,0,...,9046,9385,9668,9889,10164,10464,10700,11101,Alabama,AL
1,Alaska,0,0,0,0,0,0,0,0,0,...,374,377,378,379,379,383,383,383,Alaska,AK
2,American Samoa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,American Samoa,NaN
3,Arizona,0,0,0,0,1,1,1,1,1,...,9945,10526,10960,11119,11383,11736,12216,12674,Arizona,AZ
4,Arkansas,0,0,0,0,0,0,0,0,0,...,3703,3747,3747,4012,4043,4164,4236,4366,Arkansas,AR


In [15]:
tweets_count = Counter(df['state'])
df_date['tweets_count'] = df_date['state'].apply(lambda x: tweets_count[x])
df_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Columns: 118 entries, Province_State to tweets_count
dtypes: int64(115), object(3)
memory usage: 53.6+ KB


In [16]:
df_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Columns: 118 entries, Province_State to tweets_count
dtypes: int64(115), object(3)
memory usage: 53.6+ KB


In [17]:
cases_date = '5/8/20'
total_cases = df_date['tweets_count'].sum()
total_tweets = df_date[cases_date].sum()

## 2.2 Correlation of total tweets and cases in USA

In [18]:
tweets_date_daily = Counter(df['date'])
# drop date of 1/21/20
tweets_date_daily.pop("1/21/20", None)
date_order = sorted(tweets_date_daily)
# name format
for i in range(len(date_order)):
    x = str(date_order[i]).split('/')
    if len(x) > 1:
        if len(x[1]) == 1:
            x[1] = '0' + x[1]
            tweets_date_daily['/'.join(x)] = tweets_date_daily.pop(date_order[i])
date_order = sorted(tweets_date_daily)

In [19]:
# match date format
c = list(df_date.columns)
c_new = list(df_date.columns)
for i in range(len(c)):
    x = str(c[i]).split('/')
    if len(x) > 1:
        if len(x[1]) == 1:
            x[1] = '0' + x[1]
            c_new[i] = '/'.join(x) 
df_date.rename(columns = {c[i]:c_new[i] for i in range(len(c))}, inplace = True) 
cd = df_date.columns

In [20]:
# accumulate tweets data
tweets_date = {}
for i in range(len(date_order)):
    if i == 0:
        tweets_date[date_order[i]] = tweets_date_daily[date_order[i]]
    else:
        tweets_date[date_order[i]] = tweets_date_daily[date_order[i]] + tweets_date[date_order[i-1]]   
# total cases
cases_date = {key:df_date[key].sum() for key in tweets_date}

In [21]:
def sentiment_percentage(key, sentiment):
    return (df[(df['date'] == key)]['sentiment']==sentiment).sum()/(df[(df['date'] == key)]['sentiment']).count()
# positive sentiment count
sentiment_pos =  {key:sentiment_percentage(key, 'positive') for key in date_order}
# negative sentiment count
sentiment_neg =  {key:sentiment_percentage(key, 'negative') for key in date_order}
# neutral sentiment count
sentiment_neut =  {key:sentiment_percentage(key, 'neutral') for key in date_order}

In [22]:
d = {
    'date':date_order, 
    'tweets':[tweets_date[i] for i in date_order], 
    'cases':[cases_date[i] for i in date_order],
    'sentiment_pos':[sentiment_pos[i] for i in date_order],
    'sentiment_neg':[sentiment_neg[i] for i in date_order],
    'sentiment_neut':[sentiment_neut[i] for i in date_order],
}
df_tweets_cases = pd.DataFrame(data=d)
df_tweets_cases.head()

,date,tweets,cases,sentiment_pos,sentiment_neg,sentiment_neut
0,1/22/20,5070,1,0.296055,0.464694,0.239250
1,1/23/20,29788,1,0.200461,0.604782,0.194757
2,1/24/20,104594,2,0.298399,0.432706,0.268896
3,1/25/20,189463,2,0.259459,0.480446,0.260095
4,1/26/20,297655,5,0.240009,0.437999,0.321992


## Integrate data

In [23]:
def data_extract(single_state):
    df_single_state = df[df['state']==single_state]
    df_date_single_state = df_date[df_date['Province_State']==single_state]
    tweets_count_single_state = Counter(df_single_state['state'])
    df_date_single_state['tweets_count'] = df_date_single_state['state'].apply(lambda x: tweets_count_single_state[x])

    tweets_date_daily_single_state = Counter(df_single_state['date'])
    # drop date of 1/21/20
    tweets_date_daily_single_state.pop("1/21/20", None)
    date_order_single_state = sorted(tweets_date_daily_single_state)

    # date name format
    for i in range(len(date_order_single_state)):
        x = str(date_order_single_state[i]).split('/')
        if len(x) > 1:
            if len(x[1]) == 1:
                x[1] = '0' + x[1]
                tweets_date_daily_single_state['/'.join(x)] = tweets_date_daily_single_state.pop(date_order_single_state[i])
    # print(tweets_date_daily_single_state)
    date_order_single_state = sorted(tweets_date_daily_single_state)

    def sentiment_percentage(key, sentiment):
        return (df_single_state[(df_single_state['date'] == key)]['sentiment']==sentiment).sum()/(df_single_state[(df_single_state['date'] == key)]['sentiment']).count()
    # positive sentiment count
    sentiment_pos_single_state =  {key:sentiment_percentage(key, 'positive') for key in date_order_single_state}
    # negative sentiment count
    sentiment_neg_single_state =  {key:sentiment_percentage(key, 'negative') for key in date_order_single_state}
    # neutral sentiment count
    sentiment_neut_single_state =  {key:sentiment_percentage(key, 'neutral') for key in date_order_single_state}


    # accumulate tweets data
    tweets_date_single_state = {}
    for i in range(len(date_order_single_state)):
        if i == 0:
            tweets_date_single_state[date_order_single_state[i]] = tweets_date_daily_single_state[date_order_single_state[i]]
        else:
            tweets_date_single_state[date_order_single_state[i]] = tweets_date_daily_single_state[date_order_single_state[i]] + tweets_date_single_state[date_order_single_state[i-1]]   
    # total cases
    cases_date_single_state = {key:df_date_single_state[key].sum() for key in tweets_date_single_state}
    
    
    tweet = [tweets_date_single_state[i] for i in date_order_single_state] 
    case = [cases_date_single_state[i] for i in date_order_single_state]
#     'sentiment_pos':[sentiment_pos_single_state[i] for i in date_order_single_state],
#     'sentiment_neg':[sentiment_neg_single_state[i] for i in date_order_single_state],
#     'sentiment_neut':[sentiment_neut_single_state[i] for i in date_order_single_state],
#     sentiment = [sentiment_pos_single_state[i]-sentiment_neg_single_state[i] for i in date_order_single_state]
    sentiment = [sentiment_pos_single_state[i] for i in date_order_single_state]
    
    return tweet, case, sentiment




In [24]:
# Organize dataset for us
d_overall = {
    'date':d['date'], 
    'case_us':d['cases'],
    'tweet_us':d['tweets'], 
    'sentiment_us':[p-n for p, n in zip(d['sentiment_pos'], d['sentiment_neg'])]
}
df_overall = pd.DataFrame(data=d_overall)
df_overall



,date,case_us,tweet_us,sentiment_us
0,1/22/20,1,5070,-0.168639
1,1/23/20,1,29788,-0.404321
2,1/24/20,2,104594,-0.134307
3,1/25/20,2,189463,-0.220988
4,1/26/20,5,297655,-0.197991
...,...,...,...,...
102,5/04/20,1180375,13601803,-0.106193
103,5/05/20,1204351,13724102,-0.105504
104,5/06/20,1229331,13848764,-0.101089
105,5/07/20,1257023,13967566,-0.079519


In [25]:
states_full_dic = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    # 'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    # 'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    # 'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    # 'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    # 'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [26]:
# dictionary of tweet, case, sentiment for each state
df_tweets_all = dict()
df_cases_all = dict()
df_sentiment_all = dict()

for single_state, single_state_abb in states_full_dic.items():
    print("Analyzing at state: ", single_state)
    tweet, case, sentiment = data_extract(single_state)
    df_cases_all['case_%s'%single_state_abb] = case
    df_tweets_all['tweet_%s'%single_state_abb] = tweet
    df_sentiment_all['sentiment_%s'%single_state_abb] = sentiment
    

Analyzing at state:  Alabama


/Users/allen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Analyzing at state:  Alaska
Analyzing at state:  Arizona
Analyzing at state:  Arkansas
Analyzing at state:  California
Analyzing at state:  Colorado
Analyzing at state:  Connecticut
Analyzing at state:  Delaware
Analyzing at state:  District of Columbia
Analyzing at state:  Florida
Analyzing at state:  Georgia
Analyzing at state:  Hawaii
Analyzing at state:  Idaho
Analyzing at state:  Illinois
Analyzing at state:  Indiana
Analyzing at state:  Iowa
Analyzing at state:  Kansas
Analyzing at state:  Kentucky
Analyzing at state:  Louisiana
Analyzing at state:  Maine
Analyzing at state:  Maryland
Analyzing at state:  Massachusetts
Analyzing at state:  Michigan
Analyzing at state:  Minnesota
Analyzing at state:  Mississippi
Analyzing at state:  Missouri
Analyzing at state:  Montana
Analyzing at state:  Nebraska
Analyzing at state:  Nevada
Analyzing at state:  New Hampshire
Analyzing at state:  New Jersey
Analyzing at state:  New Mexico
Analyzing at state:  New York
Analyzing at state:  North 

In [27]:
print(len(df_tweets_all))
print(len(df_cases_all))
print(len(df_sentiment_all))

51
51
51


In [28]:
for key, values in df_cases_all.items():
    if len(values) == 0:
        print("%s, %i " % (key, len(values)))

In [29]:
df_case_overall = pd.DataFrame(data=df_cases_all)
df_tweet_overall = pd.DataFrame(data=df_tweets_all)
df_sentiment_overall = pd.DataFrame(data=df_sentiment_all)

df_state_overall = pd.concat([df_case_overall, 
                              df_tweet_overall.reindex(df_case_overall.index), 
                              df_sentiment_overall.reindex(df_case_overall.index)], 
                              axis=1)
df_state_overall

,case_AL,case_AK,case_AZ,case_AR,case_CA,case_CO,case_CT,case_DE,case_DC,case_FL,...,sentiment_SD,sentiment_TN,sentiment_TX,sentiment_UT,sentiment_VT,sentiment_VA,sentiment_WA,sentiment_WV,sentiment_WI,sentiment_WY
0,0,0,0,0,0,0,0,0,0,0,...,0.222222,0.231707,0.277136,0.277778,0.142857,0.344000,0.333333,0.192308,0.238806,0.000000
1,0,0,0,0,0,0,0,0,0,0,...,0.146341,0.194969,0.211911,0.200000,0.200000,0.226537,0.222973,0.227723,0.130952,0.235294
2,0,0,0,0,0,0,0,0,0,0,...,0.213115,0.263448,0.351668,0.327217,0.227273,0.292881,0.296911,0.281588,0.233540,0.235669
3,0,0,0,0,0,0,0,0,0,0,...,0.189610,0.214580,0.271312,0.272941,0.336735,0.247265,0.250954,0.231579,0.230110,0.173611
4,0,0,1,0,2,0,0,0,0,0,...,0.200000,0.237152,0.235047,0.242308,0.315476,0.222452,0.260310,0.199405,0.195076,0.224806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,8112,370,8924,3491,55884,16918,29973,5288,5170,36897,...,0.370130,0.301741,0.303100,0.337920,0.308370,0.321111,0.337854,0.328090,0.297806,0.182432
103,8437,371,9305,3525,58456,17367,30621,5371,5322,37439,...,0.270270,0.327532,0.299859,0.339056,0.372093,0.318182,0.351742,0.326882,0.340609,0.324138
104,8691,372,9707,3611,60616,17832,30995,5778,5461,38002,...,0.362963,0.306027,0.319140,0.339674,0.301115,0.330003,0.346877,0.322785,0.339077,0.293103
105,9046,374,9945,3703,62148,18370,31784,5939,5654,38828,...,0.274074,0.360198,0.325467,0.379950,0.363636,0.353698,0.359840,0.352273,0.336690,0.461078


In [30]:

df_overall_data =  pd.concat([df_overall, df_state_overall.reindex(df_overall.index)], axis=1)
df_overall_data

,date,case_us,tweet_us,sentiment_us,case_AL,case_AK,case_AZ,case_AR,case_CA,case_CO,...,sentiment_SD,sentiment_TN,sentiment_TX,sentiment_UT,sentiment_VT,sentiment_VA,sentiment_WA,sentiment_WV,sentiment_WI,sentiment_WY
0,1/22/20,1,5070,-0.168639,0,0,0,0,0,0,...,0.222222,0.231707,0.277136,0.277778,0.142857,0.344000,0.333333,0.192308,0.238806,0.000000
1,1/23/20,1,29788,-0.404321,0,0,0,0,0,0,...,0.146341,0.194969,0.211911,0.200000,0.200000,0.226537,0.222973,0.227723,0.130952,0.235294
2,1/24/20,2,104594,-0.134307,0,0,0,0,0,0,...,0.213115,0.263448,0.351668,0.327217,0.227273,0.292881,0.296911,0.281588,0.233540,0.235669
3,1/25/20,2,189463,-0.220988,0,0,0,0,0,0,...,0.189610,0.214580,0.271312,0.272941,0.336735,0.247265,0.250954,0.231579,0.230110,0.173611
4,1/26/20,5,297655,-0.197991,0,0,1,0,2,0,...,0.200000,0.237152,0.235047,0.242308,0.315476,0.222452,0.260310,0.199405,0.195076,0.224806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,5/04/20,1180375,13601803,-0.106193,8112,370,8924,3491,55884,16918,...,0.370130,0.301741,0.303100,0.337920,0.308370,0.321111,0.337854,0.328090,0.297806,0.182432
103,5/05/20,1204351,13724102,-0.105504,8437,371,9305,3525,58456,17367,...,0.270270,0.327532,0.299859,0.339056,0.372093,0.318182,0.351742,0.326882,0.340609,0.324138
104,5/06/20,1229331,13848764,-0.101089,8691,372,9707,3611,60616,17832,...,0.362963,0.306027,0.319140,0.339674,0.301115,0.330003,0.346877,0.322785,0.339077,0.293103
105,5/07/20,1257023,13967566,-0.079519,9046,374,9945,3703,62148,18370,...,0.274074,0.360198,0.325467,0.379950,0.363636,0.353698,0.359840,0.352273,0.336690,0.461078


In [31]:
#output data
df_overall_data.to_csv('csv/ensemble/us_case_tweet_sentiment_analysis.csv', index=False)

In [32]:
# seperate data:
# sentiment
d_sentiments_all_state = dict()
for key, value in zip(states_full_dic.values(), df_sentiment_all.values()):
    d_sentiments_all_state[key] = value
df_sentiments_all_state_one = pd.DataFrame(data=d_sentiments_all_state)


df_sentiments_all_states = df_overall
df_sentiments_all_states = df_sentiments_all_states[['date', 'sentiment_us']]

df_sentiments_all_states_out =  pd.concat([df_sentiments_all_states, df_sentiments_all_state_one.reindex(df_overall.index)], axis=1)

#output data
df_sentiments_all_states_out.to_csv('csv/ensemble/us_sentiments_analysis.csv', index=False)
